In [24]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
import re
from langchain_core.output_parsers import JsonOutputParser
import pandas as pd
import chromadb
import uuid

In [2]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.0,
    groq_api_key="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",
)

In [34]:
loader = WebBaseLoader("https://higher.gs.com/roles/137812")
page_data = loader.load().pop().page_content
print(page_data)

Software Engineer, Corporate Planning & Management, Platform Engineering | Goldman SachsSoftware Engineer, Corporate Planning & Management, Platform Engineeringlocation_onNew York, New York, United StatesSoftware Engineer, Corporate Planning & Management, Platform EngineeringApplySoftware Engineer, Corporate Planning & Management, Platform Engineeringlocation_onNew York, New York, United StatesApply
What We Do 

Corporate Planning & Management (CPM) manages the firm’s liquidity, capital, and risk, and provides the overall financial control and reporting function. CPM unifies Finance & Planning, Spend Management, Operational Risk and Resilience, and CPM Engineering teams to deliver business planning and analytics, expense management, third party risk management, sustainability strategy for our operations and supply chain, and governance strategies across the firm. 
Within CPM, Platform Engineering provides solutions that enable the firm to manage third-party spend, data and automation, 

In [17]:
def remove_whitespace(page_data):
    # Remove extra spaces, newlines, and tabs
    cleaned_data = re.sub(r'\s+', ' ', page_data)  # Replace multiple whitespace with a single space
    cleaned_data = cleaned_data.strip()  # Remove leading and trailing spaces
    
    return cleaned_data

In [35]:
cleaned_page_data = remove_whitespace(page_data)
print(cleaned_page_data)

Software Engineer, Corporate Planning & Management, Platform Engineering | Goldman SachsSoftware Engineer, Corporate Planning & Management, Platform Engineeringlocation_onNew York, New York, United StatesSoftware Engineer, Corporate Planning & Management, Platform EngineeringApplySoftware Engineer, Corporate Planning & Management, Platform Engineeringlocation_onNew York, New York, United StatesApply What We Do Corporate Planning & Management (CPM) manages the firm’s liquidity, capital, and risk, and provides the overall financial control and reporting function. CPM unifies Finance & Planning, Spend Management, Operational Risk and Resilience, and CPM Engineering teams to deliver business planning and analytics, expense management, third party risk management, sustainability strategy for our operations and supply chain, and governance strategies across the firm. Within CPM, Platform Engineering provides solutions that enable the firm to manage third-party spend, data and automation, pla

In [8]:
prompt_extract = PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE:{page_data}
    The scraped text is from career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing following keys:
    'role', 'experience', 'skills', and 'description'
    only return the valid JSON.
    ###VALID JSON (NO PREAMBLE)
    """
)

In [36]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': cleaned_page_data})
print(res.content)

```json
{
  "role": "Software Engineer, Corporate Planning & Management, Platform Engineering",
  "experience": "2-6 years",
  "skills": [
    "Advanced degree in Computer Science, Mathematics, Electrical Engineering, or related technical discipline",
    "Experience in designing highly scalable, efficient systems",
    "Experience with microservice architectures (SOA)",
    "Experience with cloud technologies, preferably AWS or Azure",
    "Experience with SQL databases (PostgreSQL/Oracle/Sybase)",
    "Experience with Kafka, MongoDB, Hadoop, Cassandra",
    "Experience in Financial Services or FinTech",
    "Familiarity with financial markets and assets"
  ],
  "description": "Corporate Planning & Management (CPM) manages the firm’s liquidity, capital, and risk, and provides the overall financial control and reporting function. CPM unifies Finance & Planning, Spend Management, Operational Risk and Resilience, and CPM Engineering teams to deliver business planning and analytics, expen

In [37]:
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer, Corporate Planning & Management, Platform Engineering',
 'experience': '2-6 years',
 'skills': ['Advanced degree in Computer Science, Mathematics, Electrical Engineering, or related technical discipline',
  'Experience in designing highly scalable, efficient systems',
  'Experience with microservice architectures (SOA)',
  'Experience with cloud technologies, preferably AWS or Azure',
  'Experience with SQL databases (PostgreSQL/Oracle/Sybase)',
  'Experience with Kafka, MongoDB, Hadoop, Cassandra',
  'Experience in Financial Services or FinTech',
  'Familiarity with financial markets and assets'],
 'description': "Corporate Planning & Management (CPM) manages the firm’s liquidity, capital, and risk, and provides the overall financial control and reporting function. CPM unifies Finance & Planning, Spend Management, Operational Risk and Resilience, and CPM Engineering teams to deliver business planning and analytics, expense management, third party risk mana

In [20]:
df = pd.read_csv("my_portfolio.csv")

In [21]:
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [25]:
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

C:\Users\91801\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 79.3M/79.3M [01:52<00:00, 738kiB/s]


In [38]:
job = json_res
job['skills']

['Advanced degree in Computer Science, Mathematics, Electrical Engineering, or related technical discipline',
 'Experience in designing highly scalable, efficient systems',
 'Experience with microservice architectures (SOA)',
 'Experience with cloud technologies, preferably AWS or Azure',
 'Experience with SQL databases (PostgreSQL/Oracle/Sybase)',
 'Experience with Kafka, MongoDB, Hadoop, Cassandra',
 'Experience in Financial Services or FinTech',
 'Familiarity with financial markets and assets']

In [39]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [40]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

In [41]:
chain_email = prompt_email | llm
res_email = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res_email.content)

Subject: Expert Software Engineering Solutions for Corporate Planning & Management

Dear Hiring Manager,

I came across the job description for a Software Engineer in Corporate Planning & Management, Platform Engineering, and I believe AtliQ can provide the expertise and solutions you need to drive your business forward.

At AtliQ, we specialize in developing scalable, efficient systems using microservice architectures and cloud technologies such as AWS and Azure. Our team has extensive experience in designing and implementing solutions using SQL databases (PostgreSQL/Oracle/Sybase), Kafka, MongoDB, Hadoop, and Cassandra.

Our expertise in Financial Services and FinTech, combined with our knowledge of financial markets and assets, makes us an ideal partner for your Corporate Planning & Management needs. We understand the importance of providing business planning and analytics, expense management, third-party risk management, and governance strategies that align with your firm's strateg